In [1]:
# example of a dcgan on cifar10
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from matplotlib import pyplot
import keras

Using TensorFlow backend.


In [2]:
undersample_path = '../sampling/undersample/'
x_train_sub = np.load(undersample_path + 'X' + '9' + 'Prop' + '0.1' + '.npy')
y_train_sub = np.load(undersample_path + 'Y' + '9' + 'Prop' + '0.1' + '.npy')

minority_class_x = []
for i in range(len(y_train_sub)):
    if y_train_sub[i][0] == 9:
        minority_class_x.append(x_train_sub[i])
        
minority_class_x = np.array(minority_class_x)

In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	# convert from unsigned ints to floats
	X = minority_class_x.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10000, n_batch=64):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 1000 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)
# 		if i in [500,1000,2000,3000,4000,5000,6000,8000,10000]:
# 			g_model.save_weights('../model/brownlee-generator' + str(i) +  '.h5')
# 			d_model.save_weights('../model/brownlee-discriminator' + str(i) +  '.h5')

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/7, d1=0.688, d2=0.695 g=0.692
>1, 2/7, d1=0.631, d2=0.697 g=0.690
>1, 3/7, d1=0.570, d2=0.703 g=0.684
>1, 4/7, d1=0.470, d2=0.718 g=0.671
>1, 5/7, d1=0.368, d2=0.751 g=0.644
>1, 6/7, d1=0.258, d2=0.821 g=0.604
>1, 7/7, d1=0.171, d2=0.909 g=0.578
>2, 1/7, d1=0.126, d2=0.918 g=0.610
>2, 2/7, d1=0.155, d2=0.825 g=0.686
>2, 3/7, d1=0.132, d2=0.695 g=0.815
>2, 4/7, d1=0.066, d2=0.578 g=0.946
>2, 5/7, d1=0.122, d2=0.543 g=0.980
>2, 6/7, d1=0.051, d2=0.549 g=0.937
>2, 7/7, d1=0.082, d2=0.582 g=0.862
>3, 1/7, d1=0.062, d2=0.623 g=0.807
>3, 2/7, d1=0.060, d2=0.672 g=0.774
>3, 3/7, d1=0.045, d2=0.719 g=0.753
>3, 4/7, d1=0.081, d2=0.843 g=0.695
>3, 5/7, d1=0.033, d2=1.135 g=0.581
>3, 6/7, d1=0.042, d2=1.358 g=0.571
>3, 7/7, d1=0.150, d2=1.124 g=0.691
>4, 1/7, d1=0.147, d2=0.748 g=0.979
>4, 2/7, d1=0.237, d2=0.453 g=1.354
>4, 3/7, d1=0.231, d2=0.387 g=1.361
>4, 4/7, d1=0.177, d2=0.434 g=1.152
>4, 5/7, d1=0.096, d2=0.500 g=0.971
>4, 6/7, d1=0.053, d2=0.555 g=0.867
>4, 7/7, d1=0.102, d2=0.595 

>31, 6/7, d1=0.494, d2=0.319 g=3.158
>31, 7/7, d1=0.310, d2=0.069 g=3.112
>32, 1/7, d1=0.562, d2=0.259 g=2.879
>32, 2/7, d1=0.243, d2=0.152 g=3.542
>32, 3/7, d1=0.325, d2=0.219 g=3.811
>32, 4/7, d1=0.357, d2=0.160 g=3.549
>32, 5/7, d1=0.343, d2=0.565 g=5.430
>32, 6/7, d1=0.397, d2=0.035 g=5.782
>32, 7/7, d1=0.390, d2=0.051 g=3.905
>33, 1/7, d1=0.422, d2=1.179 g=8.326
>33, 2/7, d1=1.433, d2=0.461 g=5.557
>33, 3/7, d1=0.547, d2=0.587 g=5.145
>33, 4/7, d1=0.763, d2=0.199 g=3.695
>33, 5/7, d1=0.691, d2=1.420 g=3.276
>33, 6/7, d1=1.141, d2=0.276 g=2.947
>33, 7/7, d1=0.931, d2=0.491 g=2.177
>34, 1/7, d1=0.416, d2=0.329 g=2.653
>34, 2/7, d1=0.733, d2=0.249 g=2.280
>34, 3/7, d1=0.806, d2=0.357 g=1.985
>34, 4/7, d1=0.622, d2=0.311 g=2.021
>34, 5/7, d1=0.344, d2=0.386 g=2.267
>34, 6/7, d1=0.528, d2=0.233 g=2.512
>34, 7/7, d1=0.465, d2=0.170 g=2.467
>35, 1/7, d1=0.730, d2=0.296 g=2.296
>35, 2/7, d1=0.533, d2=0.243 g=2.028
>35, 3/7, d1=0.216, d2=0.310 g=2.475
>35, 4/7, d1=0.361, d2=0.301 g=2.604
>

>63, 6/7, d1=0.698, d2=0.605 g=1.327
>63, 7/7, d1=0.587, d2=0.662 g=1.515
>64, 1/7, d1=0.878, d2=0.500 g=1.472
>64, 2/7, d1=0.687, d2=0.500 g=1.564
>64, 3/7, d1=0.579, d2=0.336 g=1.842
>64, 4/7, d1=0.649, d2=0.356 g=1.907
>64, 5/7, d1=0.593, d2=0.317 g=1.983
>64, 6/7, d1=0.515, d2=0.316 g=1.969
>64, 7/7, d1=0.447, d2=0.286 g=1.754
>65, 1/7, d1=0.401, d2=0.372 g=1.881
>65, 2/7, d1=0.531, d2=0.412 g=1.978
>65, 3/7, d1=0.499, d2=0.362 g=1.919
>65, 4/7, d1=0.398, d2=0.430 g=1.781
>65, 5/7, d1=0.431, d2=0.383 g=1.789
>65, 6/7, d1=0.426, d2=0.386 g=1.828
>65, 7/7, d1=0.390, d2=0.441 g=1.654
>66, 1/7, d1=0.566, d2=0.465 g=1.577
>66, 2/7, d1=0.606, d2=0.550 g=1.358
>66, 3/7, d1=0.590, d2=0.628 g=1.397
>66, 4/7, d1=0.509, d2=0.522 g=1.465
>66, 5/7, d1=0.711, d2=0.686 g=1.226
>66, 6/7, d1=0.609, d2=0.649 g=1.232
>66, 7/7, d1=0.800, d2=0.875 g=1.060
>67, 1/7, d1=0.761, d2=0.631 g=1.033
>67, 2/7, d1=0.605, d2=0.642 g=1.121
>67, 3/7, d1=0.613, d2=0.581 g=1.097
>67, 4/7, d1=0.441, d2=0.563 g=1.132
>

>95, 4/7, d1=0.358, d2=0.415 g=1.476
>95, 5/7, d1=0.417, d2=0.389 g=1.492
>95, 6/7, d1=0.445, d2=0.441 g=1.426
>95, 7/7, d1=0.442, d2=0.470 g=1.352
>96, 1/7, d1=0.491, d2=0.487 g=1.413
>96, 2/7, d1=0.331, d2=0.423 g=1.389
>96, 3/7, d1=0.430, d2=0.532 g=1.572
>96, 4/7, d1=0.449, d2=0.479 g=1.482
>96, 5/7, d1=0.605, d2=0.561 g=1.677
>96, 6/7, d1=0.529, d2=0.437 g=1.621
>96, 7/7, d1=0.605, d2=0.561 g=1.720
>97, 1/7, d1=0.633, d2=0.402 g=1.984
>97, 2/7, d1=0.762, d2=0.442 g=1.552
>97, 3/7, d1=0.602, d2=0.566 g=1.576
>97, 4/7, d1=0.739, d2=0.454 g=1.672
>97, 5/7, d1=0.768, d2=0.604 g=1.492
>97, 6/7, d1=0.695, d2=0.526 g=1.475
>97, 7/7, d1=0.777, d2=0.528 g=1.441
>98, 1/7, d1=0.786, d2=0.557 g=1.377
>98, 2/7, d1=0.644, d2=0.463 g=1.472
>98, 3/7, d1=0.544, d2=0.455 g=1.519
>98, 4/7, d1=0.560, d2=0.380 g=1.538
>98, 5/7, d1=0.575, d2=0.441 g=1.569
>98, 6/7, d1=0.625, d2=0.603 g=1.654
>98, 7/7, d1=0.701, d2=0.452 g=1.552
>99, 1/7, d1=0.685, d2=0.470 g=1.405
>99, 2/7, d1=0.615, d2=0.492 g=1.480
>

>126, 6/7, d1=0.487, d2=0.428 g=1.795
>126, 7/7, d1=0.605, d2=0.401 g=1.705
>127, 1/7, d1=0.559, d2=0.526 g=1.550
>127, 2/7, d1=0.429, d2=0.465 g=1.678
>127, 3/7, d1=0.639, d2=0.559 g=1.630
>127, 4/7, d1=0.557, d2=0.350 g=1.511
>127, 5/7, d1=0.501, d2=0.484 g=1.587
>127, 6/7, d1=0.543, d2=0.369 g=1.543
>127, 7/7, d1=0.463, d2=0.515 g=1.515
>128, 1/7, d1=0.509, d2=0.451 g=1.482
>128, 2/7, d1=0.584, d2=0.544 g=1.335
>128, 3/7, d1=0.494, d2=0.478 g=1.410
>128, 4/7, d1=0.656, d2=0.458 g=1.365
>128, 5/7, d1=0.547, d2=0.567 g=1.340
>128, 6/7, d1=0.551, d2=0.545 g=1.378
>128, 7/7, d1=0.493, d2=0.449 g=1.605
>129, 1/7, d1=0.603, d2=0.454 g=1.457
>129, 2/7, d1=0.588, d2=0.545 g=1.224
>129, 3/7, d1=0.517, d2=0.564 g=1.278
>129, 4/7, d1=0.416, d2=0.553 g=1.381
>129, 5/7, d1=0.441, d2=0.487 g=1.445
>129, 6/7, d1=0.503, d2=0.434 g=1.454
>129, 7/7, d1=0.590, d2=0.532 g=1.502
>130, 1/7, d1=0.473, d2=0.443 g=1.567
>130, 2/7, d1=0.379, d2=0.458 g=1.548
>130, 3/7, d1=0.349, d2=0.523 g=1.598
>130, 4/7, d

...

>5653, 6/7, d1=0.000, d2=0.003 g=13.237
>5653, 7/7, d1=0.001, d2=0.017 g=12.588
>5654, 1/7, d1=0.000, d2=0.009 g=12.929
>5654, 2/7, d1=0.060, d2=0.062 g=20.448
>5654, 3/7, d1=0.005, d2=0.000 g=23.095
>5654, 4/7, d1=0.020, d2=0.007 g=20.506
>5654, 5/7, d1=0.012, d2=0.008 g=19.286
>5654, 6/7, d1=0.000, d2=0.010 g=18.526
>5654, 7/7, d1=0.003, d2=0.000 g=16.582
>5655, 1/7, d1=0.014, d2=0.082 g=29.356
>5655, 2/7, d1=0.012, d2=0.000 g=34.726
>5655, 3/7, d1=0.625, d2=0.000 g=19.017
>5655, 4/7, d1=0.001, d2=0.000 g=11.481
>5655, 5/7, d1=0.000, d2=0.481 g=54.348
>5655, 6/7, d1=0.091, d2=0.000 g=73.305
>5655, 7/7, d1=0.029, d2=0.000 g=70.863
>5656, 1/7, d1=0.226, d2=0.000 g=42.017
>5656, 2/7, d1=0.184, d2=0.001 g=20.365
>5656, 3/7, d1=0.013, d2=0.006 g=11.742
>5656, 4/7, d1=0.001, d2=0.008 g=8.345
>5656, 5/7, d1=0.000, d2=0.087 g=16.369
>5656, 6/7, d1=0.021, d2=0.000 g=20.380
>5656, 7/7, d1=0.140, d2=0.000 g=18.951
>5657, 1/7, d1=0.000, d2=0.000 g=17.773
>5657, 2/7, d1=0.000, d2=0.000 g=14.577
>

>5683, 3/7, d1=0.000, d2=0.000 g=15.814
>5683, 4/7, d1=0.012, d2=0.000 g=13.003
>5683, 5/7, d1=0.075, d2=0.004 g=9.012
>5683, 6/7, d1=0.001, d2=0.076 g=18.918
>5683, 7/7, d1=0.001, d2=0.000 g=26.276
>5684, 1/7, d1=0.000, d2=0.000 g=25.815
>5684, 2/7, d1=0.003, d2=0.000 g=23.112
>5684, 3/7, d1=0.042, d2=0.000 g=18.474
>5684, 4/7, d1=0.000, d2=0.000 g=14.376
>5684, 5/7, d1=0.000, d2=0.001 g=10.520
>5684, 6/7, d1=0.001, d2=0.049 g=14.168
>5684, 7/7, d1=0.000, d2=0.000 g=16.390
>5685, 1/7, d1=0.000, d2=0.000 g=14.228
>5685, 2/7, d1=0.082, d2=0.002 g=10.177
>5685, 3/7, d1=0.000, d2=0.017 g=11.899
>5685, 4/7, d1=0.007, d2=0.009 g=13.602
>5685, 5/7, d1=0.000, d2=0.000 g=11.894
>5685, 6/7, d1=0.003, d2=0.191 g=30.805
>5685, 7/7, d1=0.137, d2=0.000 g=40.698
>5686, 1/7, d1=0.004, d2=0.000 g=37.663
>5686, 2/7, d1=0.087, d2=0.000 g=26.233
>5686, 3/7, d1=0.112, d2=0.000 g=19.694
>5686, 4/7, d1=0.056, d2=0.000 g=12.151
>5686, 5/7, d1=0.000, d2=0.019 g=11.604
>5686, 6/7, d1=0.000, d2=0.005 g=13.611
>

>5712, 7/7, d1=0.000, d2=0.022 g=15.814
>5713, 1/7, d1=0.000, d2=0.000 g=13.847
>5713, 2/7, d1=0.000, d2=0.000 g=11.240
>5713, 3/7, d1=0.000, d2=0.006 g=9.356
>5713, 4/7, d1=0.000, d2=0.082 g=13.488
>5713, 5/7, d1=0.006, d2=0.000 g=16.203
>5713, 6/7, d1=0.000, d2=0.000 g=15.789
>5713, 7/7, d1=0.000, d2=0.002 g=13.437
>5714, 1/7, d1=0.002, d2=0.135 g=15.048
>5714, 2/7, d1=0.014, d2=0.000 g=15.772
>5714, 3/7, d1=0.041, d2=0.000 g=13.644
>5714, 4/7, d1=0.001, d2=0.022 g=13.356
>5714, 5/7, d1=0.002, d2=0.000 g=14.108
>5714, 6/7, d1=0.001, d2=0.000 g=11.158
>5714, 7/7, d1=0.005, d2=0.102 g=17.575
>5715, 1/7, d1=0.111, d2=0.000 g=19.084
>5715, 2/7, d1=0.013, d2=0.000 g=16.818
>5715, 3/7, d1=0.013, d2=0.000 g=14.185
>5715, 4/7, d1=0.001, d2=0.000 g=12.477
>5715, 5/7, d1=0.012, d2=0.008 g=9.851
>5715, 6/7, d1=0.001, d2=0.088 g=17.421
>5715, 7/7, d1=0.003, d2=0.000 g=21.388
>5716, 1/7, d1=0.078, d2=0.000 g=18.946
>5716, 2/7, d1=0.001, d2=0.000 g=16.420
>5716, 3/7, d1=0.000, d2=0.000 g=13.838
>5

>5742, 4/7, d1=0.239, d2=0.000 g=11.985
>5742, 5/7, d1=0.000, d2=0.001 g=9.519
>5742, 6/7, d1=0.000, d2=0.117 g=14.889
>5742, 7/7, d1=0.000, d2=0.000 g=17.539
>5743, 1/7, d1=0.158, d2=0.000 g=14.229
>5743, 2/7, d1=0.085, d2=0.001 g=10.268
>5743, 3/7, d1=0.000, d2=0.052 g=14.844
>5743, 4/7, d1=0.001, d2=0.000 g=16.733
>5743, 5/7, d1=0.003, d2=0.000 g=15.358
>5743, 6/7, d1=0.114, d2=0.001 g=11.044
>5743, 7/7, d1=0.001, d2=0.004 g=8.742
>5744, 1/7, d1=0.000, d2=0.088 g=15.177
>5744, 2/7, d1=0.266, d2=0.000 g=15.215
>5744, 3/7, d1=0.004, d2=0.000 g=13.355
>5744, 4/7, d1=0.001, d2=0.008 g=11.507
>5744, 5/7, d1=0.018, d2=0.003 g=9.087
>5744, 6/7, d1=0.034, d2=0.016 g=9.716
>5744, 7/7, d1=0.134, d2=0.118 g=22.479
>5745, 1/7, d1=0.000, d2=0.000 g=31.570
>5745, 2/7, d1=0.075, d2=0.000 g=28.015
>5745, 3/7, d1=0.324, d2=0.000 g=20.708
>5745, 4/7, d1=0.000, d2=0.000 g=16.993
>5745, 5/7, d1=0.000, d2=0.000 g=12.753
>5745, 6/7, d1=0.028, d2=0.000 g=9.412
>5745, 7/7, d1=0.042, d2=0.039 g=9.763
>5746,

>5772, 1/7, d1=0.003, d2=0.002 g=10.799
>5772, 2/7, d1=0.000, d2=0.019 g=11.540
>5772, 3/7, d1=0.011, d2=0.001 g=9.181
>5772, 4/7, d1=0.000, d2=0.084 g=17.305
>5772, 5/7, d1=0.005, d2=0.000 g=20.589
>5772, 6/7, d1=0.027, d2=0.000 g=17.201
>5772, 7/7, d1=0.160, d2=0.000 g=11.901
>5773, 1/7, d1=0.076, d2=0.037 g=12.929
>5773, 2/7, d1=0.002, d2=0.000 g=13.118
>5773, 3/7, d1=0.007, d2=0.001 g=10.695
>5773, 4/7, d1=0.000, d2=0.006 g=10.295
>5773, 5/7, d1=0.004, d2=0.134 g=17.231
>5773, 6/7, d1=0.004, d2=0.000 g=22.581
>5773, 7/7, d1=0.219, d2=0.000 g=16.212
>5774, 1/7, d1=0.000, d2=0.000 g=12.251
>5774, 2/7, d1=0.000, d2=0.009 g=11.607
>5774, 3/7, d1=0.000, d2=0.006 g=10.213
>5774, 4/7, d1=0.000, d2=0.100 g=13.869
>5774, 5/7, d1=0.001, d2=0.000 g=14.472
>5774, 6/7, d1=0.011, d2=0.007 g=12.815
>5774, 7/7, d1=0.000, d2=0.090 g=22.397
>5775, 1/7, d1=0.009, d2=0.000 g=26.820
>5775, 2/7, d1=0.000, d2=0.000 g=24.386
>5775, 3/7, d1=0.034, d2=0.000 g=18.580
>5775, 4/7, d1=0.109, d2=0.000 g=14.490
>